In [1]:
!pip install rake-nltk

  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7821 sha256=06f336b675c962f4e658b0a887bd4e796b2edad3d994f315e6fb46a8b594a5dc
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk


In [2]:
!pip install nltk

In [3]:
import json
from rake_nltk import Rake
import pickle
import pandas as pd
import re
import os

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Reading Data

In [5]:
if os.path.isfile('/content/drive/My Drive/Data/business.pkl'):
    print ("Business pickle exists. Reading pkl")

    file = open('/content/drive/My Drive/Data/business.pkl', 'rb')
    business = pickle.load(file)
else:
    print('Reading json. Saving pkl')
    b = open('/content/drive/My Drive/Data/business.json',encoding="utf8")
    business = pd.DataFrame([json.loads(x) for x in b.readlines()])

    file = open('/content/drive/My Drive/Data/business.pkl', 'wb')
    pickle.dump(business, file)
    file.close()

Business pickle exists. Reading pkl


In [6]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


Filtering businesses.
Keeping only business that :


*   Contain **food** or **restaurant** as category
*   Are still open
*   Have received more than 50 reviews
*   Are from Nevada state





In [0]:
relevant_business = business[business['categories'].str.lower().str.contains('food|restaurants') == True]
relevant_business = relevant_business[relevant_business['is_open'] == 1]
relevant_business = relevant_business[relevant_business['review_count'] > 50]
relevant_business = relevant_business[relevant_business['state'] == 'NV']
# relevant_business.head()

In [0]:
relevant_business = relevant_business.sort_values(by= 'review_count', ascending=True)
top_15_business = relevant_business.head(15)

In [9]:
top_15_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
32586,8RGoNUEC4QGbpF4af7CDdQ,Little Caesar's,"3545 S Fort Apache Rd, Unit 115",Las Vegas,NV,89147,36.125120,-115.297975,3.0,51,1,"{'WiFi': ''no'', 'RestaurantsGoodForGroups': '...","Fast Food, Restaurants, Pizza","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'..."
134909,3p7mUvZtYocYf8gYEENGxw,Simon & Joe's Eatery,"3720 W Tropicana Ave, Ste 3",Las Vegas,NV,89103,36.101732,-115.189169,4.0,51,1,"{'BusinessParking': '{'garage': False, 'street...","Restaurants, Bakeries, Pizza, Food, Mediterran...","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'..."
28920,c_3abcaptUG0d4A4LWMe7w,Carl's Jr.,6949 N Durango Dr,Las Vegas,NV,89149,36.286254,-115.288156,2.5,51,1,"{'WiFi': 'u'no'', 'BusinessParking': '{'garage...","Fast Food, Restaurants",None
15877,5n2dA_QqO4c_7pSK0Of6AQ,Fatburger,2845 S Nellis Blvd,Las Vegas,NV,89121,36.137864,-115.065947,2.5,51,1,"{'RestaurantsReservations': 'False', 'OutdoorS...","Restaurants, American (Traditional), Food, Fas...","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'..."
93161,WUswpeAjbiww_xkSS8jTEA,Pupusas La Familiar Restaurant,4524 E Tropicana Ave,Las Vegas,NV,89121,36.100836,-115.075063,3.0,51,1,"{'GoodForKids': 'True', 'RestaurantsGoodForGro...","Mexican, Burgers, Latin American, Breakfast & ...","{'Monday': '12:0-21:0', 'Tuesday': '9:0-21:0',..."


In [10]:
if os.path.isfile('/content/drive/My Drive/Data/review.pkl'):
    print ("Review review pickle exists. Reading pkl")

    file = open('/content/drive/My Drive/Data/review.pkl', 'rb')
    review_data = pickle.load(file)
else:
    print('Reading review json. Saving pkl')
    r = open('/content/drive/My Drive/Data/review.json',encoding="utf8")
    review_data = pd.DataFrame([json.loads(x) for x in r.readlines()])

    file = open('/content/drive/My Drive/Data/review.pkl', 'wb')
    pickle.dump(review_data, file)
    file.close()

Review review pickle exists. Reading pkl


In [11]:
review_data.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38


In [12]:
if os.path.isfile('/content/drive/My Drive/Data/users.pkl'):
    print ("Review users pickle exists. Reading pkl")

    file = open('/content/drive/My Drive/Data/users.pkl', 'rb')
    users = pickle.load(file)
else:
    print('Reading user json. Saving pkl')
    u = open('/content/drive/My Drive/Data/user.json',encoding="utf8")
    users = pd.DataFrame([json.loads(x) for x in u.readlines()])

    file = open('/content/drive/My Drive/Data/users.pkl', 'wb')
    pickle.dump(users, file)
    file.close()

Review users pickle exists. Reading pkl


In [13]:
users.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,l6BmjZMeQD3rDxWUbiAiow,Rashmi,95,2013-10-08 23:11:33,84,17,25,"2015,2016,2017","c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g...",5,4.03,2,0,0,0,0,1,1,1,1,2,0
1,4XChL029mKr5hydo79Ljxg,Jenna,33,2013-02-21 22:29:06,48,22,16,,"kEBTgDvFX754S68FllfCaA, aB2DynOxNOJK9st2ZeGTPg...",4,3.63,1,0,0,0,0,0,0,1,1,0,0
2,bc8C_eETBWL0olvFSJJd0w,David,16,2013-10-04 00:16:10,28,8,10,,"4N-HU_T32hLENLntsNKNBg, pSY2vwWLgWfGVAAiKQzMng...",0,3.71,0,0,0,0,0,1,0,0,0,0,0
3,dD0gZpBctWGdWo9WlGuhlA,Angela,17,2014-05-22 15:57:30,30,4,14,,"RZ6wS38wnlXyj-OOdTzBxA, l5jxZh1KsgI8rMunm-GN6A...",5,4.85,1,0,0,0,0,0,2,0,0,1,0
4,MM4RJAeH6yuaN8oZDSt0RA,Nancy,361,2013-10-23 07:02:50,1114,279,665,"2015,2016,2017,2018","mbwrZ-RS76V1HoJ0bF_Geg, g64lOV39xSLRZO0aQQ6DeQ...",39,4.08,28,1,0,0,1,16,57,80,80,25,5


In [14]:
len(set(users['user_id']))

1637138

In [0]:
# final_Data = relevant_business.join(review_data.set_index('business_id'), on='business_id', lsuffix='_caller', rsuffix='_other')
final_Data = top_15_business.join(review_data.set_index('business_id'), on='business_id', lsuffix='_caller', rsuffix='_other')

In [0]:
final_Data.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_caller,review_count,is_open,attributes,categories,hours,review_id,user_id,stars_other,useful,funny,cool,text,date
32586,8RGoNUEC4QGbpF4af7CDdQ,Little Caesar's,"3545 S Fort Apache Rd, Unit 115",Las Vegas,NV,89147,36.12512,-115.297975,3.0,51,1,"{'WiFi': ''no'', 'RestaurantsGoodForGroups': '...","Fast Food, Restaurants, Pizza","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",94Xk1JLan_wOKPMwPiol8A,22laZkhv3jlWIEttT6HU1g,1.0,0,0,0,Pizza was delicious ...I ordered a stuffed cru...,2018-08-23 22:08:24
32586,8RGoNUEC4QGbpF4af7CDdQ,Little Caesar's,"3545 S Fort Apache Rd, Unit 115",Las Vegas,NV,89147,36.12512,-115.297975,3.0,51,1,"{'WiFi': ''no'', 'RestaurantsGoodForGroups': '...","Fast Food, Restaurants, Pizza","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",aKCanvjC3yCkIgD0v-yIjA,4TOydrfx1__SgB-pFvXVCg,2.0,0,2,1,The pizza is not the best but edible if you ar...,2016-07-15 02:59:51
32586,8RGoNUEC4QGbpF4af7CDdQ,Little Caesar's,"3545 S Fort Apache Rd, Unit 115",Las Vegas,NV,89147,36.12512,-115.297975,3.0,51,1,"{'WiFi': ''no'', 'RestaurantsGoodForGroups': '...","Fast Food, Restaurants, Pizza","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",HaYNQt6gM41e9iceXz0vaw,-wECSFoZMlaIQdV89AjOYQ,5.0,0,0,0,Thank you to the staff for still having pizza ...,2017-07-19 04:58:49
32586,8RGoNUEC4QGbpF4af7CDdQ,Little Caesar's,"3545 S Fort Apache Rd, Unit 115",Las Vegas,NV,89147,36.12512,-115.297975,3.0,51,1,"{'WiFi': ''no'', 'RestaurantsGoodForGroups': '...","Fast Food, Restaurants, Pizza","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",Maz1dNaNFcmNOz6UJ91x7Q,hv9pA78bSScx5_cBUt0uoA,5.0,2,0,0,I don't think I've ever gotten a cold pizza fr...,2017-10-03 11:55:24
32586,8RGoNUEC4QGbpF4af7CDdQ,Little Caesar's,"3545 S Fort Apache Rd, Unit 115",Las Vegas,NV,89147,36.12512,-115.297975,3.0,51,1,"{'WiFi': ''no'', 'RestaurantsGoodForGroups': '...","Fast Food, Restaurants, Pizza","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",BFFfbJuORXLGdX8g_I3j0A,7s8sVNCXnzsPNjBTqQO0WQ,5.0,0,0,0,Great price for very decent pizzas!! But this ...,2017-06-29 08:44:06


Preprocessing data by :


*   Lemmatizing words
*   Converting string into lower case
*   Removing stop words
*   Removing special characters and numbers



In [0]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

In [0]:
def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])

In [0]:
def remove_special_characters(text):
  final_str = re.sub('[^a-zA-Z.\s]', '', text)
  return final_str

In [0]:
join_all_reviews = lambda a: " / ".join(a) 
merged_reviews = final_Data.groupby(by='business_id').agg({'text': join_all_reviews}).reset_index()

In [0]:
merged_reviews['lowered_string'] = merged_reviews.text.str.lower()

In [0]:
merged_reviews['without_stop_words'] = merged_reviews['lowered_string'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))

In [0]:
merged_reviews['lemmatized_string'] = merged_reviews.without_stop_words.apply(lemmatize_text)

In [0]:
merged_reviews['lemmatized_string'] = merged_reviews.lemmatized_string.apply(remove_special_characters)

# Comparing various Keyphrase Extraction methods

## Using RAKE

In [0]:
r = Rake(max_length=3)

In [0]:
rake_output = []
for text in merged_reviews['lemmatized_string']:
  r.extract_keywords_from_text(text)

  output = r.get_ranked_phrases()
  rake_output.append(output)

In [0]:
merged_reviews['rake_output'] = rake_output

In [28]:
merged_reviews.head()

,business_id,text,lowered_string,without_stop_words,lemmatized_string,rake_output
0,3-IoSBy9vRKP2MWZwlf6wA,Hard to find a good diner/greasy spoon in Vega...,hard to find a good diner/greasy spoon in vega...,hard find good diner/greasy spoon vegas food a...,hard find good dinergreasy spoon vega food alw...,"[well thought would, wait staff friendly, unfo..."
1,3p7mUvZtYocYf8gYEENGxw,In a quiet strip mall just off of the main Str...,in a quiet strip mall just off of the main str...,quiet strip mall main strip delightful restaur...,quiet strip mall main strip delightful restaur...,"[went vega exhibition, ton garlic love, sugges..."
2,5j7BnXXvlS69uLVHrY9Upw,This is a great place recently remodeled and g...,this is a great place recently remodeled and g...,great place recently remodeled great service g...,great place recently remodeled great service g...,"[tortilla hard chewy, today quite different, t..."
3,5n2dA_QqO4c_7pSK0Of6AQ,I was told so many great things about this pla...,i was told so many great things about this pla...,told many great things place entire experience...,told many great thing place entire experience ...,"[would eat lunch, wanted leave immediately, tr..."
4,8RGoNUEC4QGbpF4af7CDdQ,Pizza was delicious ...I ordered a stuffed cru...,pizza was delicious ...i ordered a stuffed cru...,pizza delicious ...i ordered stuffed crust w e...,pizza delicious ...i ordered stuffed crust w e...,"[went take trips, walked pm ross, totally tell..."


## Using PKE

In [29]:
!pip install git+https://github.com/boudinfl/pke.git

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-is7cc6wz
  Running command git clone -q https://github.com/boudinfl/pke.git /tmp/pip-req-build-is7cc6wz
     |████████████████████████████████| 245kB 2.7MB/s 
  Created wheel for pke: filename=pke-1.8.1-cp36-none-any.whl size=8754519 sha256=46d29fe90ba4c74eb29e4a80ac0e2a781b5e488bb69d94915f1b01436d6cd1ae
  Stored in directory: /tmp/pip-ephem-wheel-cache-8kf5io0t/wheels/8d/24/54/6582e854e9e32dd6c632af6762b3a5d2f6b181c2992e165462
Successfully built pke


In [0]:
import pke

In [0]:
pke_output = []
count = 0
for text in merged_reviews['lemmatized_string']:
  text = text[0 : 1000000]  
  extractor = pke.unsupervised.TopicRank()

# load the content of the document, here document is expected to be in raw
# format (i.e. a simple text file) and preprocessing is carried out using spacy
  extractor.load_document(text, language='en')

  extractor.candidate_selection()
  extractor.candidate_weighting()

  output = extractor.get_n_best(n=30)

  final_tags = [tag[0] for tag in output]
  
  pke_output.append(final_tags)

In [0]:
merged_reviews['pke_output'] = pke_output

In [33]:
merged_reviews.head()

,business_id,text,lowered_string,without_stop_words,lemmatized_string,rake_output,pke_output
0,3-IoSBy9vRKP2MWZwlf6wA,Hard to find a good diner/greasy spoon in Vega...,hard to find a good diner/greasy spoon in vega...,hard find good diner/greasy spoon vegas food a...,hard find good dinergreasy spoon vega food alw...,"[well thought would, wait staff friendly, unfo...","[friendly food, good place, good service, buff..."
1,3p7mUvZtYocYf8gYEENGxw,In a quiet strip mall just off of the main Str...,in a quiet strip mall just off of the main str...,quiet strip mall main strip delightful restaur...,quiet strip mall main strip delightful restaur...,"[went vega exhibition, ton garlic love, sugges...","[wrong pizza, places, good pizza fair price, s..."
2,5j7BnXXvlS69uLVHrY9Upw,This is a great place recently remodeled and g...,this is a great place recently remodeled and g...,great place recently remodeled great service g...,great place recently remodeled great service g...,"[tortilla hard chewy, today quite different, t...","[el pollo yucko, double check order, drier chi..."
3,5n2dA_QqO4c_7pSK0Of6AQ,I was told so many great things about this pla...,i was told so many great things about this pla...,told many great things place entire experience...,told many great thing place entire experience ...,"[would eat lunch, wanted leave immediately, tr...","[burger, option good, food good way greasy, or..."
4,8RGoNUEC4QGbpF4af7CDdQ,Pizza was delicious ...I ordered a stuffed cru...,pizza was delicious ...i ordered a stuffed cru...,pizza delicious ...i ordered stuffed crust w e...,pizza delicious ...i ordered stuffed crust w e...,"[went take trips, walked pm ross, totally tell...","[pizzas, st time, food, specific order repeat,..."


## Using scoring_keyphrases_by_ngrams_and_textrank

In [0]:
def extract_candidate_words(text, good_tags=set(['JJ','JJR','JJS','NN','NNP','NNS','NNPS'])):
    import itertools, string

    punctions = set(string.punctuation)
    stop_words = set(nltk.corpus.stopwords.words('english'))
    
    tagged_words = itertools.chain.from_iterable(nltk.pos_tag_sents(nltk.word_tokenize(sent)
                                                                    for sent in nltk.sent_tokenize(text)))
    candidates_words = [word.lower() for word, tag in tagged_words
                        if tag in good_tags and word.lower() not in stop_words
                        and not all(char in punctions for char in word)]

    return candidates_words

In [0]:
def scoring_keyphrases_by_ngrams_and_textrank(text, n_keywords=0.05):
    from itertools import takewhile, tee
    import networkx, nltk
    
    words = [word.lower()
             for sent in nltk.sent_tokenize(text)
             for word in nltk.word_tokenize(sent)]

    candidates_words = extract_candidate_words(text)
    
    #Building graph
    graph = networkx.Graph()
    graph.add_nodes_from(set(candidates_words))

    #Creating and iterating over word-pairs
    def pairwise(iterable):
        word1, word2 = tee(iterable)
        next(word2, None)
        return zip(word1, word2)

    for w1, w2 in pairwise(candidates_words):
        if w2:
            graph.add_edge(*sorted([w1, w2]))

    #Scoring nodes using default pagerank algorithm
    ranks = networkx.pagerank(graph)

    if 0 < n_keywords < 1:
        n_keywords = int(round(len(candidates_words) * n_keywords))
    word_ranks = {word_rank[0]: word_rank[1]
                  for word_rank in sorted(ranks.items(), key=lambda x: x[1], reverse=True)[:n_keywords]}
    keywords = set(word_ranks.keys())

    # networkx.draw(graph, with_labels=True)
    # plt.show()

    
    #Forming keyphrases from keywords
    keyphrases = {}
    j = 0
    for i, word in enumerate(words):
        if i < j:
            continue
        if word in keywords:
            kp_words = list(takewhile(lambda x: x in keywords, words[i:i+10]))
            avg_pagerank = sum(word_ranks[w] for w in kp_words) / float(len(kp_words))
            keyphrases[' '.join(kp_words)] = avg_pagerank
            j = i + len(kp_words)
    
    sorted_answer = sorted(keyphrases.items(), key=lambda x: x[1], reverse=True)

    final_answer = []
    check_tag_present = {}
    for tags in sorted_answer:
      temp = tags[0].split()
      
      sorted_temp = sorted(temp)
      sorted_temp_str = ' '.join([t for t in sorted_temp])
      if len(sorted_temp) > 1 and not check_tag_present.get(sorted_temp_str):
        final_answer.append(tags[0])
        check_tag_present[sorted_temp_str] = 1
        
    return final_answer

In [0]:
tks_output = []
count = 0

for text in merged_reviews['lemmatized_string']:
  output = scoring_keyphrases_by_ngrams_and_textrank(text)
  tks_output.append(output)

In [0]:
merged_reviews['tks_output'] = tks_output

In [39]:
merged_reviews.head()

,business_id,text,lowered_string,without_stop_words,lemmatized_string,rake_output,pke_output,tks_output
0,3-IoSBy9vRKP2MWZwlf6wA,Hard to find a good diner/greasy spoon in Vega...,hard to find a good diner/greasy spoon in vega...,hard find good diner/greasy spoon vegas food a...,hard find good dinergreasy spoon vega food alw...,"[well thought would, wait staff friendly, unfo...","[friendly food, good place, good service, buff...","[buffet food, good food, good place, good food..."
1,3p7mUvZtYocYf8gYEENGxw,In a quiet strip mall just off of the main Str...,in a quiet strip mall just off of the main str...,quiet strip mall main strip delightful restaur...,quiet strip mall main strip delightful restaur...,"[went vega exhibition, ton garlic love, sugges...","[wrong pizza, places, good pizza fair price, s...","[pizza place, good pizza, great pizza food, pi..."
2,5j7BnXXvlS69uLVHrY9Upw,This is a great place recently remodeled and g...,this is a great place recently remodeled and g...,great place recently remodeled great service g...,great place recently remodeled great service g...,"[tortilla hard chewy, today quite different, t...","[el pollo yucko, double check order, drier chi...","[chicken bowl, chicken salad, chicken taco, ti..."
3,5n2dA_QqO4c_7pSK0Of6AQ,I was told so many great things about this pla...,i was told so many great things about this pla...,told many great things place entire experience...,told many great thing place entire experience ...,"[would eat lunch, wanted leave immediately, tr...","[burger, option good, food good way greasy, or...","[food burger, burger place, good burger, burge..."
4,8RGoNUEC4QGbpF4af7CDdQ,Pizza was delicious ...I ordered a stuffed cru...,pizza was delicious ...i ordered a stuffed cru...,pizza delicious ...i ordered stuffed crust w e...,pizza delicious ...i ordered stuffed crust w e...,"[went take trips, walked pm ross, totally tell...","[pizzas, st time, food, specific order repeat,...","[pizza location, pizza time, good pizza, pizza..."


## Using PyTextRank

In [73]:
!pip install pytextrank

In [0]:
import spacy
import pytextrank

nlp = spacy.load("en_core_web_sm")
nlp.max_length = 5000000

# add PyTextRank to the spaCy pipeline
tr = pytextrank.TextRank()
nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)

In [0]:
textrank_output = []
count = 0
for text in merged_reviews['lemmatized_string']:
  doc = nlp(text)

  all_phrases = []  
  for p in doc._.phrases:
    all_phrases.append(p.text)

  output = all_phrases
  textrank_output.append(output)

In [0]:
merged_reviews['pytextrank'] = textrank_output

In [77]:
merged_reviews.head()

,business_id,text,lowered_string,without_stop_words,lemmatized_string,rake_output,pke_output,tks_output,pytextrank
0,3-IoSBy9vRKP2MWZwlf6wA,Hard to find a good diner/greasy spoon in Vega...,hard to find a good diner/greasy spoon in vega...,hard find good diner/greasy spoon vegas food a...,hard find good dinergreasy spoon vega food alw...,"[well thought would, wait staff friendly, unfo...","[friendly food, good place, good service, buff...","[buffet food, good food, good place, good food...","[good food good service, food buffet, food buf..."
1,3p7mUvZtYocYf8gYEENGxw,In a quiet strip mall just off of the main Str...,in a quiet strip mall just off of the main str...,quiet strip mall main strip delightful restaur...,quiet strip mall main strip delightful restaur...,"[went vega exhibition, ton garlic love, sugges...","[wrong pizza, places, good pizza fair price, s...","[pizza place, good pizza, great pizza food, pi...","[good pizza service, great pizza food, times p..."
2,5j7BnXXvlS69uLVHrY9Upw,This is a great place recently remodeled and g...,this is a great place recently remodeled and g...,great place recently remodeled great service g...,great place recently remodeled great service g...,"[tortilla hard chewy, today quite different, t...","[el pollo yucko, double check order, drier chi...","[chicken bowl, chicken salad, chicken taco, ti...","[fast food chicken, chicken avocado bowl chick..."
3,5n2dA_QqO4c_7pSK0Of6AQ,I was told so many great things about this pla...,i was told so many great things about this pla...,told many great things place entire experience...,told many great thing place entire experience ...,"[would eat lunch, wanted leave immediately, tr...","[burger, option good, food good way greasy, or...","[food burger, burger place, good burger, burge...","[burger fries, burger place, good fix burger, ..."
4,8RGoNUEC4QGbpF4af7CDdQ,Pizza was delicious ...I ordered a stuffed cru...,pizza was delicious ...i ordered a stuffed cru...,pizza delicious ...i ordered stuffed crust w e...,pizza delicious ...i ordered stuffed crust w e...,"[went take trips, walked pm ross, totally tell...","[pizzas, st time, food, specific order repeat,...","[pizza location, pizza time, good pizza, pizza...","[pizza time, pizza hot ready fresh pizza littl..."


# Generating output

In [0]:
merged_reviews = merged_reviews.drop(['text', 'lowered_string', 'without_stop_words', 'lemmatized_string'], axis=1)

In [79]:
merged_reviews.head()

,business_id,rake_output,pke_output,tks_output,pytextrank
0,3-IoSBy9vRKP2MWZwlf6wA,"[well thought would, wait staff friendly, unfo...","[friendly food, good place, good service, buff...","[buffet food, good food, good place, good food...","[good food good service, food buffet, food buf..."
1,3p7mUvZtYocYf8gYEENGxw,"[went vega exhibition, ton garlic love, sugges...","[wrong pizza, places, good pizza fair price, s...","[pizza place, good pizza, great pizza food, pi...","[good pizza service, great pizza food, times p..."
2,5j7BnXXvlS69uLVHrY9Upw,"[tortilla hard chewy, today quite different, t...","[el pollo yucko, double check order, drier chi...","[chicken bowl, chicken salad, chicken taco, ti...","[fast food chicken, chicken avocado bowl chick..."
3,5n2dA_QqO4c_7pSK0Of6AQ,"[would eat lunch, wanted leave immediately, tr...","[burger, option good, food good way greasy, or...","[food burger, burger place, good burger, burge...","[burger fries, burger place, good fix burger, ..."
4,8RGoNUEC4QGbpF4af7CDdQ,"[went take trips, walked pm ross, totally tell...","[pizzas, st time, food, specific order repeat,...","[pizza location, pizza time, good pizza, pizza...","[pizza time, pizza hot ready fresh pizza littl..."


In [0]:
export_csv = merged_reviews.to_csv (r'/content/drive/My Drive/Data/Review_50.csv', index = None, header=True)

In [0]:
business_tags = final_Data.join(merged_reviews.set_index('business_id'), on='business_id', lsuffix='_caller', rsuffix='_other')

In [82]:
business_tags.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_caller,review_count,is_open,attributes,categories,hours,review_id,user_id,stars_other,useful,funny,cool,text,date,rake_output,pke_output,tks_output,pytextrank
32586,8RGoNUEC4QGbpF4af7CDdQ,Little Caesar's,"3545 S Fort Apache Rd, Unit 115",Las Vegas,NV,89147,36.12512,-115.297975,3.0,51,1,"{'WiFi': ''no'', 'RestaurantsGoodForGroups': '...","Fast Food, Restaurants, Pizza","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",94Xk1JLan_wOKPMwPiol8A,22laZkhv3jlWIEttT6HU1g,1.0,0,0,0,Pizza was delicious ...I ordered a stuffed cru...,2018-08-23 22:08:24,"[went take trips, walked pm ross, totally tell...","[pizzas, st time, food, specific order repeat,...","[pizza location, pizza time, good pizza, pizza...","[pizza time, pizza hot ready fresh pizza littl..."
32586,8RGoNUEC4QGbpF4af7CDdQ,Little Caesar's,"3545 S Fort Apache Rd, Unit 115",Las Vegas,NV,89147,36.12512,-115.297975,3.0,51,1,"{'WiFi': ''no'', 'RestaurantsGoodForGroups': '...","Fast Food, Restaurants, Pizza","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",aKCanvjC3yCkIgD0v-yIjA,4TOydrfx1__SgB-pFvXVCg,2.0,0,2,1,The pizza is not the best but edible if you ar...,2016-07-15 02:59:51,"[went take trips, walked pm ross, totally tell...","[pizzas, st time, food, specific order repeat,...","[pizza location, pizza time, good pizza, pizza...","[pizza time, pizza hot ready fresh pizza littl..."
32586,8RGoNUEC4QGbpF4af7CDdQ,Little Caesar's,"3545 S Fort Apache Rd, Unit 115",Las Vegas,NV,89147,36.12512,-115.297975,3.0,51,1,"{'WiFi': ''no'', 'RestaurantsGoodForGroups': '...","Fast Food, Restaurants, Pizza","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",HaYNQt6gM41e9iceXz0vaw,-wECSFoZMlaIQdV89AjOYQ,5.0,0,0,0,Thank you to the staff for still having pizza ...,2017-07-19 04:58:49,"[went take trips, walked pm ross, totally tell...","[pizzas, st time, food, specific order repeat,...","[pizza location, pizza time, good pizza, pizza...","[pizza time, pizza hot ready fresh pizza littl..."
32586,8RGoNUEC4QGbpF4af7CDdQ,Little Caesar's,"3545 S Fort Apache Rd, Unit 115",Las Vegas,NV,89147,36.12512,-115.297975,3.0,51,1,"{'WiFi': ''no'', 'RestaurantsGoodForGroups': '...","Fast Food, Restaurants, Pizza","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",Maz1dNaNFcmNOz6UJ91x7Q,hv9pA78bSScx5_cBUt0uoA,5.0,2,0,0,I don't think I've ever gotten a cold pizza fr...,2017-10-03 11:55:24,"[went take trips, walked pm ross, totally tell...","[pizzas, st time, food, specific order repeat,...","[pizza location, pizza time, good pizza, pizza...","[pizza time, pizza hot ready fresh pizza littl..."
32586,8RGoNUEC4QGbpF4af7CDdQ,Little Caesar's,"3545 S Fort Apache Rd, Unit 115",Las Vegas,NV,89147,36.12512,-115.297975,3.0,51,1,"{'WiFi': ''no'', 'RestaurantsGoodForGroups': '...","Fast Food, Restaurants, Pizza","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",BFFfbJuORXLGdX8g_I3j0A,7s8sVNCXnzsPNjBTqQO0WQ,5.0,0,0,0,Great price for very decent pizzas!! But this ...,2017-06-29 08:44:06,"[went take trips, walked pm ross, totally tell...","[pizzas, st time, food, specific order repeat,...","[pizza location, pizza time, good pizza, pizza...","[pizza time, pizza hot ready fresh pizza littl..."


In [0]:
business_review_users_tags = business_tags.join(users.set_index('user_id'), on='user_id', lsuffix='_caller', rsuffix='_other')

In [84]:
business_review_users_tags.head()

,business_id,name_caller,address,city,state,postal_code,latitude,longitude,stars_caller,review_count_caller,is_open,attributes,categories,hours,review_id,user_id,stars_other,useful_caller,funny_caller,cool_caller,text,date,rake_output,pke_output,tks_output,pytextrank,name_other,review_count_other,yelping_since,useful_other,funny_other,cool_other,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
32586,8RGoNUEC4QGbpF4af7CDdQ,Little Caesar's,"3545 S Fort Apache Rd, Unit 115",Las Vegas,NV,89147,36.12512,-115.297975,3.0,51,1,"{'WiFi': ''no'', 'RestaurantsGoodForGroups': '...","Fast Food, Restaurants, Pizza","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",94Xk1JLan_wOKPMwPiol8A,22laZkhv3jlWIEttT6HU1g,1.0,0,0,0,Pizza was delicious ...I ordered a stuffed cru...,2018-08-23 22:08:24,"[went take trips, walked pm ross, totally tell...","[pizzas, st time, food, specific order repeat,...","[pizza location, pizza time, good pizza, pizza...","[pizza time, pizza hot ready fresh pizza littl...",Doll,23,2011-03-17 11:57:53,26,36,5,,"DCgYO2-K2KhyLz04nbmw3g, RY9RDYFzk2d2V6GHryIK0Q...",1,2.14,0,0,0,0,0,0,1,1,1,0,0
32586,8RGoNUEC4QGbpF4af7CDdQ,Little Caesar's,"3545 S Fort Apache Rd, Unit 115",Las Vegas,NV,89147,36.12512,-115.297975,3.0,51,1,"{'WiFi': ''no'', 'RestaurantsGoodForGroups': '...","Fast Food, Restaurants, Pizza","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",aKCanvjC3yCkIgD0v-yIjA,4TOydrfx1__SgB-pFvXVCg,2.0,0,2,1,The pizza is not the best but edible if you ar...,2016-07-15 02:59:51,"[went take trips, walked pm ross, totally tell...","[pizzas, st time, food, specific order repeat,...","[pizza location, pizza time, good pizza, pizza...","[pizza time, pizza hot ready fresh pizza littl...",Nicole,58,2009-08-18 05:46:56,94,22,22,,None,0,3.56,0,0,0,0,0,1,7,0,0,0,0
32586,8RGoNUEC4QGbpF4af7CDdQ,Little Caesar's,"3545 S Fort Apache Rd, Unit 115",Las Vegas,NV,89147,36.12512,-115.297975,3.0,51,1,"{'WiFi': ''no'', 'RestaurantsGoodForGroups': '...","Fast Food, Restaurants, Pizza","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",HaYNQt6gM41e9iceXz0vaw,-wECSFoZMlaIQdV89AjOYQ,5.0,0,0,0,Thank you to the staff for still having pizza ...,2017-07-19 04:58:49,"[went take trips, walked pm ross, totally tell...","[pizzas, st time, food, specific order repeat,...","[pizza location, pizza time, good pizza, pizza...","[pizza time, pizza hot ready fresh pizza littl...",Victoria,24,2016-09-26 02:15:29,7,8,2,,"74L-RSUi9Kx-_9rtqB0YCw, _xSjS7u16ikMg75i3rNXsw...",0,3.30,0,0,0,0,0,0,0,0,0,0,0
32586,8RGoNUEC4QGbpF4af7CDdQ,Little Caesar's,"3545 S Fort Apache Rd, Unit 115",Las Vegas,NV,89147,36.12512,-115.297975,3.0,51,1,"{'WiFi': ''no'', 'RestaurantsGoodForGroups': '...","Fast Food, Restaurants, Pizza","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",Maz1dNaNFcmNOz6UJ91x7Q,hv9pA78bSScx5_cBUt0uoA,5.0,2,0,0,I don't think I've ever gotten a cold pizza fr...,2017-10-03 11:55:24,"[went take trips, walked pm ross, totally tell...","[pizzas, st time, food, specific order repeat,...","[pizza location, pizza time, good pizza, pizza...","[pizza time, pizza hot ready fresh pizza littl...",Michael,19,2017-08-13 05:46:45,10,1,8,,"JdkzbT7Dn7es52_fzC0vrg, hABSVPL4CsiIBohNsrDlfw...",0,4.48,0,0,0,0,0,0,1,0,0,0,0
32586,8RGoNUEC4QGbpF4af7CDdQ,Little Caesar's,"3545 S Fort Apache Rd, Unit 115",Las Vegas,NV,89147,36.12512,-115.297975,3.0,51,1,"{'WiFi': ''no'', 'RestaurantsGoodForGroups': '...","Fast Food, Restaurants, Pizza","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",BFFfbJuORXLGdX8g_I3j0A,7s8sVNCXnzsPNjBTqQO0WQ,5.0,0,0,0,Great price for very decent pizzas!! But this ...,2017-06-29 08:44:06,"[went take trips, walked pm ross, totally tell...","[pizzas, st time, food, specific order repeat,...","[pizza location, pizza time, good pizza, pizza...","[pizza time, pizza hot ready fresh pizza littl...",Phng,14,2015-

In [0]:
business_review_users_tags = business_review_users_tags.drop(['address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars_caller', 'review_count_caller',
       'is_open', 'attributes', 'categories', 'hours',
       'stars_other', 'useful_caller', 'funny_caller', 'cool_caller',
       'review_count_other', 'yelping_since', 'useful_other', 'funny_other',
       'cool_other', 'elite', 'friends', 'fans', 'average_stars',
       'compliment_hot', 'compliment_more', 'compliment_profile',
       'compliment_cute', 'compliment_list', 'compliment_note',
       'compliment_plain', 'compliment_cool', 'compliment_funny',
       'compliment_writer', 'compliment_photos'], axis=1)

In [0]:
business_review_users_tags = business_review_users_tags.rename({'name_caller': 'business_name', 'name_other': 'username'}, axis=1) 

In [0]:
file = open('/content/drive/My Drive/Data/business_review_users_tags.pkl', 'wb')
pickle.dump(business_review_users_tags, file)
file.close()

In [0]:
business_review_users_tags.to_csv (r'/content/drive/My Drive/Data/business_review_users_tags.csv', index = None, header=True)

In [0]:
filter_review_tags = business_review_users_tags[['text', 'pytextrank']]
filter_review_tags.to_csv (r'/content/drive/My Drive/Data/filter_review_tags.csv', index = None, header=True)

In [0]:
compare_tags = business_review_users_tags[['text', 'pytextrank', 'tks_output']]
compare_tags.to_csv (r'/content/drive/My Drive/Data/compare_tags.csv', index = None, header=True)

In [0]:
business_tags = business_review_users_tags.drop_duplicates(subset=['business_id'])
business_tags = business_tags[['business_id', 'pytextrank', 'rake_output', 'pke_output', 'tks_output']]
business_tags.to_csv(r'/content/drive/My Drive/Data/business_tags.csv', index = None, header=True)

In [92]:
business_tags.head()

,business_id,pytextrank,rake_output,pke_output,tks_output
32586,8RGoNUEC4QGbpF4af7CDdQ,"[pizza time, pizza hot ready fresh pizza littl...","[went take trips, walked pm ross, totally tell...","[pizzas, st time, food, specific order repeat,...","[pizza location, pizza time, good pizza, pizza..."
134909,3p7mUvZtYocYf8gYEENGxw,"[good pizza service, great pizza food, times p...","[went vega exhibition, ton garlic love, sugges...","[wrong pizza, places, good pizza fair price, s...","[pizza place, good pizza, great pizza food, pi..."
28920,c_3abcaptUG0d4A4LWMe7w,"[fast food fast food place, fast food food, fo...","[world kind practice, went day buck, way treat...","[burger comfort food, mess order, love carls f...","[food food, food burger, food place, food good..."
15877,5n2dA_QqO4c_7pSK0Of6AQ,"[burger fries, burger place, good fix burger, ...","[would eat lunch, wanted leave immediately, tr...","[burger, option good, food good way greasy, or...","[food burger, burger place, good burger, burge..."
93161,WUswpeAjbiww_xkSS8jTEA,"[food place order, warm made order food, worst...","[wave nausea diarreah, today decided brunch, s...","[pupusas good, food, grubhub order, minute hit...","[order food, place food, food place order, foo..."


# Evaluation

In [93]:
!pip install easy-rouge

In [0]:
from rouge.rouge import rouge_n_sentence_level

def calcualte_score_using_rouge(summary_sentence, reference_sentence):
  recall, precision, rouge = rouge_n_sentence_level(summary_sentence, reference_sentence, 3)
  print('ROUGE-2-R', recall)
  print('ROUGE-2-P', precision)
  print('ROUGE-2-F', rouge)

  return recall, precision, rouge

In [95]:
total_recall_rake = 0
total_precision_rake = 0
total_f1_rake = 0

total_recall_pke = 0
total_precision_pke = 0
total_f1_pke = 0

total_recall_tks = 0
total_precision_tks = 0
total_f1_tks = 0


for i in range(0, len(business_tags)):
  print('-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-# ' + str(i) + ' #-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-')
  print('-----------------------RAKE-----------------------')
  summary_sentence = ' '.join(([tks for tks in business_tags.iloc[i]['rake_output']]))
  reference_sentence = ' '.join(([ptr for ptr in business_tags.iloc[i]['pytextrank']]))

  recall, precision, rouge = calcualte_score_using_rouge(summary_sentence, reference_sentence)

  total_recall_rake += recall
  total_precision_rake += precision
  total_f1_rake += rouge

  print('-----------------------PKE-----------------------')
  summary_sentence = ' '.join(([tks for tks in business_tags.iloc[i]['pke_output']]))
  reference_sentence = ' '.join(([ptr for ptr in business_tags.iloc[i]['pytextrank']]))

  recall, precision, rouge = calcualte_score_using_rouge(summary_sentence, reference_sentence)

  total_recall_pke += recall
  total_precision_pke += precision
  total_f1_pke += rouge

  print('-----------------------TKS-----------------------')
  summary_sentence = ' '.join(([tks for tks in business_tags.iloc[i]['tks_output']]))
  reference_sentence = ' '.join(([ptr for ptr in business_tags.iloc[i]['pytextrank']]))

  recall, precision, rouge = calcualte_score_using_rouge(summary_sentence, reference_sentence)

  total_recall_tks += recall
  total_precision_tks += precision
  total_f1_tks += rouge

-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-# 0 #-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
-----------------------RAKE-----------------------
ROUGE-2-R 0.16604805626098848
ROUGE-2-P 0.8559919436052367
ROUGE-2-F 0.27814136125654454
-----------------------PKE-----------------------
ROUGE-2-R 0.07560070326235593
ROUGE-2-P 0.9555555555555556
ROUGE-2-F 0.14011585807385954
-----------------------TKS-----------------------
ROUGE-2-R 0.259034967767142
ROUGE-2-P 0.9038854805725971
ROUGE-2-F 0.40267233525660495
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-# 1 #-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
-----------------------RAKE-----------------------
ROUGE-2-R 0.15964560347709797
ROUGE-2-P 0.8673932788374206
ROUGE-2-F 0.26965974869405623
-----------------------PKE-----------------------
ROUGE-2-R 0.0710464727515881
ROUGE-2-P 0.9837962962962963
ROUGE-2-F 0.13252260679763017
-----------------------TKS-----------------------
ROUGE-2-R 0.22684720829154129
ROUGE-2-P 0.9679029957203994
ROUGE-2-F 0.36755146262188515
-#-#-#-#-#-#-#-#-#-#-#-

In [0]:
total_recall_rake /= len(business_tags)
total_precision_rake /= len(business_tags)
total_f1_rake /= len(business_tags)

total_recall_pke /= len(business_tags)
total_precision_pke /= len(business_tags)
total_f1_pke /= len(business_tags)

total_recall_tks /= len(business_tags)
total_precision_tks /= len(business_tags)
total_f1_tks /= len(business_tags)

In [0]:
all_precision = [total_precision_rake, total_precision_pke, total_precision_tks]
all_recall = [total_recall_rake, total_recall_pke, total_recall_tks]
all_f1 = [total_f1_rake, total_f1_pke, total_f1_tks]

In [0]:
all_values = pd.DataFrame()

In [0]:
all_values['algorithms'] = ['RAKE', 'PKE', 'OUR APPROACH']
all_values['precision'] = all_precision
all_values['recall'] = all_recall
all_values['f1'] = all_f1

In [100]:
all_values.set_index('algorithms')

,precision,recall,f1
algorithms,,,
RAKE,0.904193,0.140130,0.241220
PKE,0.979417,0.055337,0.104462
OUR APPROACH,0.955238,0.239826,0.381065
